In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarusr
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser
from scipy.stats import entropy

from sklearn.metrics.cluster import normalized_mutual_info_score
from itertools import combinations

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_cnVICES"]="1"

In [3]:
! nvidia-smi

Wed Aug 10 17:39:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 27%   34C    P8    20W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 25%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_cn = db.lda_topic_cn
    
    
except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_cn

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'lda_topic_cn')

# CN

In [6]:
cn_topic = db_cn.aggregate([
        {"$unwind" : "$classification"}
])


In [7]:
cn_df = pd.DataFrame(cn_topic)

In [8]:
# 取cpc前3碼
cn_df["cpc_n"] = cn_df.classification.apply(lambda x: x[0:3])

In [9]:
cn_df

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c855efb892ae553bd9fc,CN102140507B,2010-12-20 05:00:00,[Nankai Univ],Y02A50/30,本发明提供了一种关于引起人类感染性腹泻的主要病原微生物的基因芯片及检测用试剂盒，其主要针对致...,1,Y02
1,62c6c855efb892ae553bd9ff,CN102080127A,2010-09-06 04:00:00,"[GUANGZHOU IMPROVE MEDICAL CO Ltd, Wuhan Unive...",Y02A50/30,本发明公开了一种检测15种临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异...,5,Y02
2,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],G06F17/00,可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G06
3,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],B60W50/087,可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,B60
4,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],G01C21/3641,可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G01
...,...,...,...,...,...,...,...,...
124985,62c6c855efb892ae553c67e5,CN111935889A,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],Y02B20/40,本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6,Y02
124986,62c6c855efb892ae553c67e7,CN111353035B,2020-03-11 04:00:00,[Mgjia Beijing Technology Co ltd],G06F16/3329,本公开涉及一种人机对话方法、装置、可读存储介质及电子设备。方法包括：在接收到用户当前输入的文...,3,G06
124987,62c6c855efb892ae553c67e7,CN111353035B,2020-03-11 04:00:00,[Mgjia Beijing Technology Co ltd],G06N3/0445,本公开涉及一种人机对话方法、装置、可读存储介质及电子设备。方法包括：在接收到用户当前输入的文...,3,G06
124988,62c6c855efb892ae553c67e7,CN111353035B,2020-03-11 04:00:00,[Mgjia Beijing Technology Co ltd],G06N3/0454,本公开涉及一种人机对话方法、装置、可读存储介质及电子设备。方法包括：在接收到用户当前输入的文...,3,G06


In [10]:
cn_df_2 = pd.DataFrame(list(db_cn.find()))

In [11]:
cn_df_2

,_id,patentNumber,appDate,company,classification,brief,topic
0,62c6c855efb892ae553bd9fb,CN102259231A,2010-05-28 04:00:00,[China Huaye Group Co Ltd],None,本发明提供一种回转窑筒体的高空焊接工艺，包括：装配能够使回转窑筒体实现分段转动的临时盘车措施...,5
1,62c6c855efb892ae553bd9fc,CN102140507B,2010-12-20 05:00:00,[Nankai Univ],[Y02A50/30],本发明提供了一种关于引起人类感染性腹泻的主要病原微生物的基因芯片及检测用试剂盒，其主要针对致...,1
2,62c6c855efb892ae553bd9fd,CN101892395A,2010-06-12 04:00:00,"[ZHEJIANG HUAGUANG SMELTING GROUP CO Ltd, Adva...",None,一种微波真空分解冶炼金属镁的方法，属于金属镁冶炼领域。首先将原料镁砂、石灰与硅铁破碎后按比例...,4
3,62c6c855efb892ae553bd9fe,CN101831505A,2010-05-21 04:00:00,[Wuhan University WHU],None,本发明公开了一种检测临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异检测探...,5
4,62c6c855efb892ae553bd9ff,CN102080127A,2010-09-06 04:00:00,"[GUANGZHOU IMPROVE MEDICAL CO Ltd, Wuhan Unive...",[Y02A50/30],本发明公开了一种检测15种临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异...,5
...,...,...,...,...,...,...,...
36330,62c6c855efb892ae553c67e5,CN111935889A,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],"[H05B47/175, H05B47/19, G01R31/00, H05B47/105,...",本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6
36331,62c6c855efb892ae553c67e6,CN212660357U,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],None,本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6
36332,62c6c855efb892ae553c67e7,CN111353035B,2020-03-11 04:00:00,[Mgjia Beijing Technology Co ltd],"[G06F16/3329, G06N3/0445, G06N3/0454, G06N3/08]",本公开涉及一种人机对话方法、装置、可读存储介质及电子设备。方法包括：在接收到用户当前输入的文...,3
36333,62c6c855efb892ae553c67e8,CN211184794U,2020-02-07 05:00:00,[Bizlink International Corp],None,本实用新型为一种具有鳍片组的连接器，为电子装置的散热模组，其包含一外壳、一电路板、一热扩散件...,6


In [12]:
# cn_df_2.classification = cn_df_2.classification.fillna(0)

In [13]:
cn_df_2 = cn_df_2[cn_df_2.classification.notna()]

In [14]:
cn_df_2.reset_index(drop=True,inplace=True)

In [15]:
cn_df_2.classification[0][0:3]

['Y02A50/30']

In [16]:
# 取cpc前3碼
cn_df_2["cpc_n"] = cn_df_2.classification.apply(lambda x: x[0][0:3])

/tmp/ipykernel_10879/434860568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cn_df_2["cpc_n"] = cn_df_2.classification.apply(lambda x: x[0][0:3])


In [17]:
cn_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c855efb892ae553bd9fc,CN102140507B,2010-12-20 05:00:00,[Nankai Univ],[Y02A50/30],本发明提供了一种关于引起人类感染性腹泻的主要病原微生物的基因芯片及检测用试剂盒，其主要针对致...,1,Y02
1,62c6c855efb892ae553bd9ff,CN102080127A,2010-09-06 04:00:00,"[GUANGZHOU IMPROVE MEDICAL CO Ltd, Wuhan Unive...",[Y02A50/30],本发明公开了一种检测15种临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异...,5,Y02
2,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],"[G06F17/00, B60W50/087, G01C21/3641, B60W2520/...",可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G06
3,62c6c855efb892ae553bda02,CN101882864B,2010-06-25 04:00:00,[Hangzhou Silergy Semiconductor Technology Ltd...,"[G05F1/10, H02M1/32, H02M1/36, H03K17/0822, H0...",一种上电启动电路及其上电启动方法，它是将功率开关管以及控制电路集成于一芯片中，并通过一外接的...,3,G05
4,62c6c855efb892ae553bda03,CN101728051B,2010-02-03 05:00:00,[Institute of Electrical Engineering of CAS],"[H01F6/04, H01F6/06]",一种大分离间隙的高磁场超导磁体系统，其超导线圈包括低温超导线圈(16)和高温超导线圈(17)...,5,H01
...,...,...,...,...,...,...,...,...
23299,62c6c855efb892ae553c67ce,CN213904899U,2020-12-16 05:00:00,[Derui Shenzhen Information Technology Co ltd],[Y02B70/10],一种物联网高效大功率电源，包括电源模块和LED背光驱动模块，所述电源模块的输出电压范围为DC...,6,Y02
23300,62c6c855efb892ae553c67e0,CN213213897U,2020-09-14 04:00:00,[NANJING PUTIAN DATANG INFORMATION ELECTRONICS...,[Y02B20/40],本实用新型的一种智慧电源，包括输入电压检测电路、输入电流检测电路、EMI模块、LED开关电源...,6,Y02
23301,62c6c855efb892ae553c67e1,CN212844086U,2020-10-09 04:00:00,"[Shandong Deer Intelligent Digital Co ltd, Jin...",[Y02D30/70],本实用新型公开了一种NB热量表采集装置，包括处理单元，还包括与所述处理单元相连接的采集模块和...,1,Y02
23302,62c6c855efb892ae553c67e5,CN111935889A,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],"[H05B47/175, H05B47/19, G01R31/00, H05B47/105,...",本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6,H05


In [18]:
cn_df2 = cn_df_2.groupby(["cpc_n","topic"],as_index=False).size()

In [19]:
cn_test = cn_df2.groupby(["topic"])

In [20]:
cn_df_3 = cn_df2.sort_values('size',ascending = False)
group = cn_df_3.groupby(['topic']).head(2)
group.sort_values('topic')

,cpc_n,topic,size
542,G06,0,56
640,H04,0,159
641,H04,1,1312
543,G06,1,443
544,G06,2,18
642,H04,2,34
545,G06,3,87
643,H04,3,149
644,H04,4,152
546,G06,4,27


In [21]:
cn_df2.sort_values('size',ascending = False).head(30)

,cpc_n,topic,size
645,H04,5,5738
547,G06,5,1907
641,H04,1,1312
646,H04,6,1188
504,G01,5,825
647,H04,7,596
665,Y02,5,582
648,H04,8,580
567,G08,5,579
543,G06,1,443


## 建立Dictionary轉換DF

In [23]:
# CPC dict
cpc_dict = {
"A61":0,
"B60":1,
"C08":2,
"G01":3,
"G05":4,
"G06":5,
"G08":6,
"H01":7,
"H02":8,
"H03":9,
"H04":10,
"H05":11,
"Y02":12
}

In [24]:
# topic dict
cn_dict={
    0:"G08",
    1:"H04",
    2:"H04",
    3:"G08",
    4:"G06",
    5:"H04",
    6:"C04",
    7:"Y02",
    8:"H04",
    9:"Y02",
}

In [25]:
cn_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c855efb892ae553bd9fc,CN102140507B,2010-12-20 05:00:00,[Nankai Univ],[Y02A50/30],本发明提供了一种关于引起人类感染性腹泻的主要病原微生物的基因芯片及检测用试剂盒，其主要针对致...,1,Y02
1,62c6c855efb892ae553bd9ff,CN102080127A,2010-09-06 04:00:00,"[GUANGZHOU IMPROVE MEDICAL CO Ltd, Wuhan Unive...",[Y02A50/30],本发明公开了一种检测15种临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异...,5,Y02
2,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],"[G06F17/00, B60W50/087, G01C21/3641, B60W2520/...",可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G06
3,62c6c855efb892ae553bda02,CN101882864B,2010-06-25 04:00:00,[Hangzhou Silergy Semiconductor Technology Ltd...,"[G05F1/10, H02M1/32, H02M1/36, H03K17/0822, H0...",一种上电启动电路及其上电启动方法，它是将功率开关管以及控制电路集成于一芯片中，并通过一外接的...,3,G05
4,62c6c855efb892ae553bda03,CN101728051B,2010-02-03 05:00:00,[Institute of Electrical Engineering of CAS],"[H01F6/04, H01F6/06]",一种大分离间隙的高磁场超导磁体系统，其超导线圈包括低温超导线圈(16)和高温超导线圈(17)...,5,H01
...,...,...,...,...,...,...,...,...
23299,62c6c855efb892ae553c67ce,CN213904899U,2020-12-16 05:00:00,[Derui Shenzhen Information Technology Co ltd],[Y02B70/10],一种物联网高效大功率电源，包括电源模块和LED背光驱动模块，所述电源模块的输出电压范围为DC...,6,Y02
23300,62c6c855efb892ae553c67e0,CN213213897U,2020-09-14 04:00:00,[NANJING PUTIAN DATANG INFORMATION ELECTRONICS...,[Y02B20/40],本实用新型的一种智慧电源，包括输入电压检测电路、输入电流检测电路、EMI模块、LED开关电源...,6,Y02
23301,62c6c855efb892ae553c67e1,CN212844086U,2020-10-09 04:00:00,"[Shandong Deer Intelligent Digital Co ltd, Jin...",[Y02D30/70],本实用新型公开了一种NB热量表采集装置，包括处理单元，还包括与所述处理单元相连接的采集模块和...,1,Y02
23302,62c6c855efb892ae553c67e5,CN111935889A,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],"[H05B47/175, H05B47/19, G01R31/00, H05B47/105,...",本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6,H05


In [26]:
cn_dict.get(cn_df_2.topic[0])

'H04'

In [27]:
# topic to CPC
cn_df_2["topic_to_cpc"] = cn_df_2.topic.apply(lambda x: cn_dict.get(x))

/tmp/ipykernel_10879/110480285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cn_df_2["topic_to_cpc"] = cn_df_2.topic.apply(lambda x: cn_dict.get(x))


In [28]:
cn_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n,topic_to_cpc
0,62c6c855efb892ae553bd9fc,CN102140507B,2010-12-20 05:00:00,[Nankai Univ],[Y02A50/30],本发明提供了一种关于引起人类感染性腹泻的主要病原微生物的基因芯片及检测用试剂盒，其主要针对致...,1,Y02,H04
1,62c6c855efb892ae553bd9ff,CN102080127A,2010-09-06 04:00:00,"[GUANGZHOU IMPROVE MEDICAL CO Ltd, Wuhan Unive...",[Y02A50/30],本发明公开了一种检测15种临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异...,5,Y02,H04
2,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],"[G06F17/00, B60W50/087, G01C21/3641, B60W2520/...",可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G06,H04
3,62c6c855efb892ae553bda02,CN101882864B,2010-06-25 04:00:00,[Hangzhou Silergy Semiconductor Technology Ltd...,"[G05F1/10, H02M1/32, H02M1/36, H03K17/0822, H0...",一种上电启动电路及其上电启动方法，它是将功率开关管以及控制电路集成于一芯片中，并通过一外接的...,3,G05,G08
4,62c6c855efb892ae553bda03,CN101728051B,2010-02-03 05:00:00,[Institute of Electrical Engineering of CAS],"[H01F6/04, H01F6/06]",一种大分离间隙的高磁场超导磁体系统，其超导线圈包括低温超导线圈(16)和高温超导线圈(17)...,5,H01,H04
...,...,...,...,...,...,...,...,...,...
23299,62c6c855efb892ae553c67ce,CN213904899U,2020-12-16 05:00:00,[Derui Shenzhen Information Technology Co ltd],[Y02B70/10],一种物联网高效大功率电源，包括电源模块和LED背光驱动模块，所述电源模块的输出电压范围为DC...,6,Y02,C04
23300,62c6c855efb892ae553c67e0,CN213213897U,2020-09-14 04:00:00,[NANJING PUTIAN DATANG INFORMATION ELECTRONICS...,[Y02B20/40],本实用新型的一种智慧电源，包括输入电压检测电路、输入电流检测电路、EMI模块、LED开关电源...,6,Y02,C04
23301,62c6c855efb892ae553c67e1,CN212844086U,2020-10-09 04:00:00,"[Shandong Deer Intelligent Digital Co ltd, Jin...",[Y02D30/70],本实用新型公开了一种NB热量表采集装置，包括处理单元，还包括与所述处理单元相连接的采集模块和...,1,Y02,H04
23302,62c6c855efb892ae553c67e5,CN111935889A,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],"[H05B47/175, H05B47/19, G01R31/00, H05B47/105,...",本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6,H05,C04


In [29]:
# cpc to num
cn_df_2["cpc_to_num"] = cn_df_2.cpc_n.apply(lambda x: cpc_dict.get(x))
cn_df_2["topic_to_num"] = cn_df_2.topic_to_cpc.apply(lambda x: cpc_dict.get(x))

/tmp/ipykernel_10879/4209253357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cn_df_2["cpc_to_num"] = cn_df_2.cpc_n.apply(lambda x: cpc_dict.get(x))
/tmp/ipykernel_10879/4209253357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cn_df_2["topic_to_num"] = cn_df_2.topic_to_cpc.apply(lambda x: cpc_dict.get(x))


In [30]:
cn_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n,topic_to_cpc,cpc_to_num,topic_to_num
0,62c6c855efb892ae553bd9fc,CN102140507B,2010-12-20 05:00:00,[Nankai Univ],[Y02A50/30],本发明提供了一种关于引起人类感染性腹泻的主要病原微生物的基因芯片及检测用试剂盒，其主要针对致...,1,Y02,H04,12.0,10.0
1,62c6c855efb892ae553bd9ff,CN102080127A,2010-09-06 04:00:00,"[GUANGZHOU IMPROVE MEDICAL CO Ltd, Wuhan Unive...",[Y02A50/30],本发明公开了一种检测15种临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异...,5,Y02,H04,12.0,10.0
2,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],"[G06F17/00, B60W50/087, G01C21/3641, B60W2520/...",可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G06,H04,5.0,10.0
3,62c6c855efb892ae553bda02,CN101882864B,2010-06-25 04:00:00,[Hangzhou Silergy Semiconductor Technology Ltd...,"[G05F1/10, H02M1/32, H02M1/36, H03K17/0822, H0...",一种上电启动电路及其上电启动方法，它是将功率开关管以及控制电路集成于一芯片中，并通过一外接的...,3,G05,G08,4.0,6.0
4,62c6c855efb892ae553bda03,CN101728051B,2010-02-03 05:00:00,[Institute of Electrical Engineering of CAS],"[H01F6/04, H01F6/06]",一种大分离间隙的高磁场超导磁体系统，其超导线圈包括低温超导线圈(16)和高温超导线圈(17)...,5,H01,H04,7.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...
23299,62c6c855efb892ae553c67ce,CN213904899U,2020-12-16 05:00:00,[Derui Shenzhen Information Technology Co ltd],[Y02B70/10],一种物联网高效大功率电源，包括电源模块和LED背光驱动模块，所述电源模块的输出电压范围为DC...,6,Y02,C04,12.0,NaN
23300,62c6c855efb892ae553c67e0,CN213213897U,2020-09-14 04:00:00,[NANJING PUTIAN DATANG INFORMATION ELECTRONICS...,[Y02B20/40],本实用新型的一种智慧电源，包括输入电压检测电路、输入电流检测电路、EMI模块、LED开关电源...,6,Y02,C04,12.0,NaN
23301,62c6c855efb892ae553c67e1,CN212844086U,2020-10-09 04:00:00,"[Shandong Deer Intelligent Digital Co ltd, Jin...",[Y02D30/70],本实用新型公开了一种NB热量表采集装置，包括处理单元，还包括与所述处理单元相连接的采集模块和...,1,Y02,H04,12.0,10.0
23302,62c6c855efb892ae553c67e5,CN111935889A,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],"[H05B47/175, H05B47/19, G01R31/00, H05B47/105,...",本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6,H05,C04,11.0,NaN


In [31]:
print(sum(cn_df_2.cpc_to_num.isna()))
print(sum(cn_df_2.topic_to_num.isna()))

3448
3080


In [32]:
# fill NA
cn_df_2.cpc_to_num.fillna(99,inplace=True)
cn_df_2.topic_to_num.fillna(99,inplace=True)

/tmp/ipykernel_10879/1525773425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cn_df_2.cpc_to_num.fillna(99,inplace=True)
/tmp/ipykernel_10879/1525773425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cn_df_2.topic_to_num.fillna(99,inplace=True)


In [33]:
cn_df_2

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n,topic_to_cpc,cpc_to_num,topic_to_num
0,62c6c855efb892ae553bd9fc,CN102140507B,2010-12-20 05:00:00,[Nankai Univ],[Y02A50/30],本发明提供了一种关于引起人类感染性腹泻的主要病原微生物的基因芯片及检测用试剂盒，其主要针对致...,1,Y02,H04,12.0,10.0
1,62c6c855efb892ae553bd9ff,CN102080127A,2010-09-06 04:00:00,"[GUANGZHOU IMPROVE MEDICAL CO Ltd, Wuhan Unive...",[Y02A50/30],本发明公开了一种检测15种临床常见病原微生物的基因芯片。该基因芯片包括固定在固相载体上的特异...,5,Y02,H04,12.0,10.0
2,62c6c855efb892ae553bda01,CN103003854B,2010-07-29 04:00:00,[Ford Global Technologies LLC],"[G06F17/00, B60W50/087, G01C21/3641, B60W2520/...",可检查车辆的动力操控状态、驾驶员到车辆的输入等来确定驾驶员工作负担的一个或多个测量。然后，可...,8,G06,H04,5.0,10.0
3,62c6c855efb892ae553bda02,CN101882864B,2010-06-25 04:00:00,[Hangzhou Silergy Semiconductor Technology Ltd...,"[G05F1/10, H02M1/32, H02M1/36, H03K17/0822, H0...",一种上电启动电路及其上电启动方法，它是将功率开关管以及控制电路集成于一芯片中，并通过一外接的...,3,G05,G08,4.0,6.0
4,62c6c855efb892ae553bda03,CN101728051B,2010-02-03 05:00:00,[Institute of Electrical Engineering of CAS],"[H01F6/04, H01F6/06]",一种大分离间隙的高磁场超导磁体系统，其超导线圈包括低温超导线圈(16)和高温超导线圈(17)...,5,H01,H04,7.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...
23299,62c6c855efb892ae553c67ce,CN213904899U,2020-12-16 05:00:00,[Derui Shenzhen Information Technology Co ltd],[Y02B70/10],一种物联网高效大功率电源，包括电源模块和LED背光驱动模块，所述电源模块的输出电压范围为DC...,6,Y02,C04,12.0,99.0
23300,62c6c855efb892ae553c67e0,CN213213897U,2020-09-14 04:00:00,[NANJING PUTIAN DATANG INFORMATION ELECTRONICS...,[Y02B20/40],本实用新型的一种智慧电源，包括输入电压检测电路、输入电流检测电路、EMI模块、LED开关电源...,6,Y02,C04,12.0,99.0
23301,62c6c855efb892ae553c67e1,CN212844086U,2020-10-09 04:00:00,"[Shandong Deer Intelligent Digital Co ltd, Jin...",[Y02D30/70],本实用新型公开了一种NB热量表采集装置，包括处理单元，还包括与所述处理单元相连接的采集模块和...,1,Y02,H04,12.0,10.0
23302,62c6c855efb892ae553c67e5,CN111935889A,2020-09-30 04:00:00,[Guilin Xintong Technology Co ltd],"[H05B47/175, H05B47/19, G01R31/00, H05B47/105,...",本发明公开了一种路灯自动采集及远程控制装置，该装置包括控制模块和分别与控制模块相连接的采集模...,6,H05,C04,11.0,99.0


# NMI計算
參考  
- [Entropy and Mutual Information](https://datascience.oneoffcoder.com/normalized-entropy-mi.html)
- [問題在Data 2: 從訊息理論來看資料的影響](https://chih-sheng-huang821.medium.com/%E7%AC%A8%E8%9B%8B-%E5%95%8F%E9%A1%8C%E5%9C%A8data-2-%E5%BE%9E%E8%A8%8A%E6%81%AF%E7%90%86%E8%AB%96%E4%BE%86%E7%9C%8B%E8%B3%87%E6%96%99%E7%9A%84%E5%BD%B1%E9%9F%BF-9aa2e2b420c6)
- [sklearn NMI](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html)


In [34]:
normalized_mutual_info_score(list(cn_df_2.cpc_to_num), list(cn_df_2.topic_to_num))

0.005618193215921658